# CFT Latency Analysis

## Setup and Helper Functions

In [ ]:
import json
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps
import biopsykit as bp
import statsmodels.api as sm

from cft_analysis.datasets import CftDatasetProcessed, CftDatasetRaw

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

pg.options["round"] = 4

palette

## Import Data

In [ ]:
# get path to analysis results
base_path = Path("..")
data_path = Path("../../2022_scientific_reports/data")

In [ ]:
results_path = base_path.joinpath("results")
stats_path = results_path.joinpath("statistics")
plot_path = results_path.joinpath("plots")
bp.utils.file_handling.mkdirs([results_path, stats_path, plot_path])

In [ ]:
dataset = CftDatasetProcessed(data_path, exclude_subjects=True)
dataset_raw = CftDatasetRaw(data_path)
dataset

In [ ]:
mdbf_data = dataset.questionnaire.filter(regex="MDBF_\w+_(pre|post)")
mdbf_data = mdbf_data.xs("CFT", level="condition")
mdbf_data = bp.questionnaires.utils.wide_to_long(mdbf_data, quest_name="MDBF", levels=["subscale", "time"])
mdbf_data = mdbf_data.unstack("time").diff(axis=1).dropna(axis=1).droplevel("time", axis=1)
mdbf_data = mdbf_data.unstack()
mdbf_data.columns = ["_".join(col) for col in mdbf_data.columns]
mdbf_data.head()

In [ ]:
cft_latency = dataset.cft_parameter.reindex(["onset_latency", "peak_brady_latency"], level="type").unstack()
data_merge = cft_latency.diff(axis=1).dropna(axis=1)["data"]
data_merge.columns = ["cft_latency"]

data_merge = data_merge.join(dataset.cortisol_features.unstack()["cortisol"]).join(mdbf_data)
data_merge = data_merge.droplevel(["condition", "subphase", "category"])

data_merge

## Bradycardia Information

In [ ]:
brady_data = dataset.cft_parameter.reindex(["baseline_hr", "peak_brady_bpm"], level="type")["data"]
brady_data = brady_data.droplevel(["subphase", "category"]).unstack()
brady_data

### CFT Latency over Time

In [ ]:
fig, ax = plt.subplots()
pg.plot_paired(data=data_merge.reset_index(), dv="cft_latency", within="phase", subject="subject", ax=ax)
fig.tight_layout()

## $CFT_{Latency}$ and Cortisol Response

In [ ]:
dv = "auc_i"

### Linear Regression with $CFT_{Latency}$ of MIST

In [ ]:
phase = "MIST3"

In [ ]:
data_analysis = data_merge.xs(phase, level="phase")
data_analysis.head()

In [ ]:
fig, ax = plt.subplots()

sns.regplot(data=data_analysis.reset_index(), x="cft_latency", y=dv, ax=ax, label=phase)
ax.legend()

fig.tight_layout()

In [ ]:
x = sm.add_constant(data_analysis["cft_latency"])
y = data_analysis[dv]

In [ ]:
res = sm.OLS(y, x).fit()
res.summary()

### Multiple Linear Regression with $CFT_{Latency}$ of all MIST Phases

In [ ]:
fig, ax = plt.subplots()

for key, data in data_merge.groupby("phase"):
    sns.regplot(data=data.reset_index(), x="cft_latency", y=dv, ax=ax, label=key)
ax.legend()
    
fig.tight_layout()

In [ ]:
x = sm.add_constant(data_merge["cft_latency"].unstack("phase"))
y = data_merge.xs("MIST1", level="phase")[dv]

In [ ]:
res = sm.OLS(y, x).fit()
res.summary()

## $CFT_{Latency}$ and Mood Response

### Linear Regression

In [ ]:
phase = "MIST1"
dv = "MDBF_AwakeTired"

In [ ]:
data_analysis = data_merge.xs(phase, level="phase")

In [ ]:
fig, ax = plt.subplots()

sns.regplot(data=data_analysis.reset_index(), x="cft_latency", y=dv, ax=ax, label=phase)
ax.legend()

fig.tight_layout()

In [ ]:
x = sm.add_constant(data_analysis["cft_latency"])
y = data_analysis[dv]

In [ ]:
res = sm.OLS(y, x).fit()
res.summary()

### Multiple Linear Regression with $CFT_{Latency}$ of MIST

In [ ]:
data_analysis = data_merge.copy()
data_analysis.head()

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(10,5))


for mood_var, ax in zip(data.filter(like="MDBF"), axs):
    legend_list = []
    for key, data in data_analysis.groupby("phase"):
        sns.regplot(data=data.reset_index(), x="cft_latency", y=mood_var, ax=ax, label=key)
    ax.legend()
    

fig.tight_layout()

In [ ]:
for mood_dv in data_merge.filter(like="MDBF"):
    x = sm.add_constant(data_merge["cft_latency"].unstack("phase"))
    y = data_merge.xs("MIST1", level="phase")[mood_dv]
    res = sm.OLS(y, x).fit()
    display(res.summary())